In [ ]:
import websocket, json

In [ ]:
!docker run -d -p 27017:27017 --name crypto_streaming mongo

In [ ]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client.test_database
collection = db.crypto_collection

In [ ]:
#collection.insert_on(message)

In [ ]:
#collection.find_one()

In [ ]:
socket = "wss://ws-feed.pro.coinbase.com"

In [ ]:
subscribe = {
    "type": "subscribe",
    "product_ids": [
        "BTC-USD",
        "ETH-USD",
        "ETH-EUR"
    ],
    "channels": [
        "level2",
        "heartbeat",
        {
            "name": "ticker",
            "product_ids": [
                "BTC-USD",
                "ETH-BTC",
                "ETH-USD"
            ]
        }
    ]
}

In [ ]:
def on_open(ws):
    print('opened')
    auth_data = subscribe
    ws.send(json.dumps(auth_data))

In [ ]:
def on_message(ws, message):
    print('received a message:')
    print(message)
    collection.insert_one(eval(message))

In [ ]:
ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message)

In [ ]:
%%time
ws.run_forever()
# 155MB in 7 minutes

In [ ]:
for item in collection.find():
    if item["type"] == "ticker":
        print(item)

In [ ]:
collection.count()

In [ ]:
collection.count({"type":"ticker"})

In [ ]:
collection.find_one({"type":"ticker"})

In [ ]:
!docker stats crypto_streaming

In [ ]:
!docker kill crypto_streaming
!docker rm crypto_streaming